<h1>Script for running our caiman analysis function on 2P data</h1>
<h3>A couple of things to note: </h3>
<ul>
  <li>Make sure that caiman is your active anaconda environment. It should say caiman (Python 3/10/9) in the top right of your VS Code notebook.</li>
  <li>If caiman is not active you can type "conda activate caiman" into the VS Code terminal at the bottom of your screen.</li>
  <li>If for some reason the script throws an error, please restart the notebook before trying to run it again.</li>
  <li>You can get paths easily on windows with shift + right click -> copy as path or on linux by dragging the file into an empty terminal and copying the resulting string.</li>
  <li>File paths on windows will need an r before the path r"myFilePath", on linux this is not necessary.</li>
  <li>You can run any cell in this notebook by either clicking the play button next to the cell, or selecting the cell and pressing ctrl + enter</li>
</ul> 

To share drive onto linux computer depending on which computer you are on the command line is:
sudo bash
mount -t cifs -o username=howelab,uid=1000,gid=1000,forceuid,forcegid //RKC-HOW-WD-0015.ad.bu.edu/LilyDrive/exp_2ptrack_2024/ /mnt/lily_shared

OR 
sudo bash
mount -t cifs -o username=howelab,uid=1002,gid=1002,forceuid,forcegid //RKC-HOW-WD-0015.ad.bu.edu/LilyDrive/exp_2ptrack_2024/ /mnt/lily_shared

In [3]:
#Importing functions from other files for use in this script
#Note that these files must be in the same directory as your script in order to import them like this
from mainCaimanFunction import runCaimanPipeline
from findDict import getCaimanParams
from tifLookup import findMovs

#assemble a list of your movies: Either create the list manually or have some function do it for you. 
#The library os with the path module (os.path) will be very usefulf for directory manipulation
# movieList = [r"G:\TdTG7_18_025\field1_track\ConcatenatedMovie_MC_reg.tif", r"D:\Ben 2P Cell Segmentation Data\Brenna Data\TdTG7_7_019\TdTG7_7_019_001_ch1_MC_reg.tif"]
#note that you need to put an r before your string (outside of the quotes) because python needs to convert the windows path to something that it can read (only need this on windows)

#You can use this function if your folder follows the pattern of mainFolder -> field folders -> videos inside each field folder
#just specify the pattern that you want the function to look for in these sub folders
# movieList = findMovs([r"G:\TdTG7_18_025", r"G:\TdTG7_18_026", r"G:\TdTG7_18_027"], 'mc_reg.tif') #-> This will return the list in the proper format, don't worry about
#formatting into a list inside a list


movieList = ["/mnt/lily_shared/tdt_011/250201/2w_rec001/ConcatenatedMovie_MC.tif"]

#Note that the internal caiman functions wants a list for each movie that you want to run. Therefore when you assemble your list make sure they are lists INSIDE of lists.
#If you need to do this (not using the findMovs function) below is an example of how to do that in python.

assembledList = []
for movie in movieList:
    assembledList.append([movie]) 
# assmebledList = [['Example.tif], ['Example2.tif']] -> Here each movie is in its own list inside the outer list
print("List of movies to process: ", assembledList)

# just loop over our assembledList and grab whichever dictionary we want, and pass that and our current fname in to the runCaimanPipeline function.
for cT in range(len(assembledList)):
    fnames = assembledList[cT]  # filename to be processed
    #note that the only required input is fnames. Give the optional input of 'brenna20' or 'gabi20' to switch between the two different param dictionaries
    paramDict = getCaimanParams(fnames, 'brenna20')

    #There are optional inputs here if you do not want to save out images of what your components look like, and if you want to save out both your bad good and bad components.
    # caimanResult = runCaimanPipeline(fnames, paramDict, selectComponents = False, saveImages = False) -> by default select components is on, and save images is on. This is how you change them
    caimanResult = runCaimanPipeline(fnames, paramDict, saveImages=False)

List of movies to process:  [['/mnt/lily_shared/tdt_011/250201/2w_rec001/ConcatenatedMovie_MC.tif']]
Defining parameters...
Starting CNMF...
Starting component evaluation....
GPU run not requested, disabling use of GPUs
USING MODEL (keras API): /home/howelab/caiman_data/model/cnn_model.json
18/18 [==============================] - 0s 16ms/step
CNMF Finished, saving images...
Time taken:51.86666666666667
Trying to remove memmap file at location: /home/howelab/caiman_data/temp/ConcatenatedMovie_MC_memmap_d1_488_d2_701_d3_1_order_C_frames_68558.mmap.


In [7]:
dview.terminate

NameError: name 'dview' is not defined

In [ ]:
from mainCaimanFunction import runCaimanPipeline
from findDict import getCaimanParams

 # filename to be processed
fnames = [r"D:\Ben 2P Cell Segmentation Data\GabiData\20x\GR_20XSample.tif"]

#I have also built in the ability to choose your dictionary and change any of the parameters that you wish as optional inputs.
#You can change any of the following parameters if you wish:

# brennaParams20x = {
# 'fnames': fnames,
#             'fr': 31,
#             'decay_time': 0.3,
#             'p': 2,
#             'nb': 2,
#             'rf': 24,
#             'K': 4, 
#             'gSig': [4,4],
#             'stride': 8,
#             'method_init': 'greedy_roi',
#             'rolling_sum': True,
#             'only_init': True,
#             'ssub': 2,
#             'tsub': 2,
#             'merge_thr': 0.60, 
#             'min_SNR': 2.0,
#             'rval_thr': 0.80,
#             'use_cnn': True,
#             'min_cnn_thr': 0.90,
#             'cnn_lowest': 0.30}

#Below is an example of how to use these optional key arguments. Just make sure you use the exact name of the key in the dictionary.
#Example: I want to change the number of components (K) you would call the function like thhis: paramDict = getCaimanParams(fnames, 'gabi20', userInput = True, K = 6)
#Note that the key argument K is not input as a string but instead as an assignment statement.
paramDict = getCaimanParams(fnames, 'gabi20', userInput = True, K = 6, ssub = 1, tsub = 1, cnn_lowest = 0.45)
#Now you can just pass that assembled parameter dictionary into the pipeline function. 
caimanResult = runCaimanPipeline(fnames, paramDict)